# ドリルの長さを調べる
### 工具の一番下に当たるピクセルを発見する

 - 工具の部分だけを切り出す
 - 2値化して、工具の部分だけ0になるようにする
 - 値が0の一番下のピクセルを探して、一番上からの距離を計算する

In [2]:
import cv2


def find_bottom_black_pixel_coordinates(image_path):
    # 画像を読み込む
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # 画像を2値化する
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)

    # 画像の高さと幅を取得
    height, width = binary_image.shape

    # 画像を下から上にスキャンして、一番最初に見つかった黒いピクセルの座標を取得
    for y in range(height-1, -1, -1):
        for x in range(width):
            if binary_image[y, x] == 0:  # 黒いピクセルを見つけたら
                marked_image = cv2.cvtColor(
                    cv2.imread(image_path), cv2.COLOR_BGR2RGB)
                marked_image[y, x] = [255, 0, 0]  # 赤い印をつける
                cv2.imwrite('./binary_image_with_marker.png',
                            cv2.cvtColor(binary_image, cv2.COLOR_GRAY2RGB))
                cv2.imwrite('./sample_images/original_image_with_marker.png',
                            cv2.cvtColor(marked_image, cv2.COLOR_RGB2BGR))

                return x, y  # 座標を返す

    # 黒いピクセルが見つからなかった場合
    return None


# 画像のパスを指定して座標を取得
image_path = './sample_images/drill_forcus_sample.png'
coordinates = find_bottom_black_pixel_coordinates(image_path)

if coordinates:
    print(f"一番下の黒いピクセルの座標: x={coordinates[0]}, y={coordinates[1]}")

else:
    print("画像中に黒いピクセルが見つかりませんでした。")

一番下の黒いピクセルの座標: x=293, y=445


# ドリルの横幅を調べる
### 工具の横幅を調べる

 - 本当に工具の種類部分だけの画像にする(別のやつが入っていると横幅に引っかかる)
 - 2値化して、工具の部分だけ0になるようにする
 - 横軸で、値が0のピクセルを探して、一番左からの距離と一番右からの距離を計算する

In [5]:
import cv2
import numpy as np


def find_max_width(image_path):
    # 画像を読み込む
    original_image = cv2.imread(image_path)
    resized_image = cv2.resize(original_image, (500, 500))  # 必要に応じてリサイズ

    # 画像を2値化する
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY)

    # 最大横幅とそのY座標を初期化
    max_width = 0
    max_width_y = 0

    target_min_x = 0
    target_max_x = 0

    # 1行ずつ黒色の最小と最大のピクセルを探し、横幅を計算する
    for y in range(binary_image.shape[0]):
        black_pixels = np.where(binary_image[y, :] == 0)[0]
        if len(black_pixels) > 0:
            min_x = np.min(black_pixels)
            max_x = np.max(black_pixels)
            width = max_x - min_x
            if width > max_width:
                target_max_x = max_x
                target_min_x = min_x
                max_width = width
                max_width_y = y

    print("最大横幅:", max_width)

    # 最大横幅があるY座標にマークを付ける
    marked_image = resized_image.copy()
    cv2.line(marked_image, (target_min_x, max_width_y),
             (target_max_x, max_width_y), (0, 255, 0), 2)
    # 画像保存
    cv2.imwrite('./marked_image_with_max_width.png', marked_image)
    print("画像を保存しました。")


# 画像のパスを指定して処理を実行
# 画像のパスを適切に指定してください
image_path = './sample_images/M3_DRILL_EXP10000_R.png'
find_max_width(image_path)

最大横幅: 499
画像を保存しました。


 画像の切り取りをして ドリル・タップの横幅の最大値を探す

In [6]:
import cv2
import numpy as np


def find_max_width(image_path):
    # 画像を読み込む
    original_image = cv2.imread(image_path)

    # 画像を反時計回りに90度回転
    rotateimg = cv2.rotate(original_image, cv2.ROTATE_90_COUNTERCLOCKWISE)

    start_y = 160
    end_y = 620 + start_y
    start_x = 280
    end_x = 380 + start_x
    # 画像を指定した範囲で切り取る
    cropped_image = rotateimg[start_y:end_y, start_x:end_x]

    # 画像を2値化する
    gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY)

    # 最大横幅とそのY座標を初期化
    max_width = 0
    max_width_y = 0

    target_min_x = 0
    target_max_x = 0

    # 1行ずつ黒色の最小と最大のピクセルを探し、横幅を計算する
    for y in range(binary_image.shape[0]):
        black_pixels = np.where(binary_image[y, :] == 0)[0]
        if len(black_pixels) > 0:
            min_x = np.min(black_pixels)
            max_x = np.max(black_pixels)
            width = max_x - min_x
            if width > max_width:
                target_max_x = max_x
                target_min_x = min_x
                max_width = width
                max_width_y = y

    print("最大横幅:", max_width)

    # 最大横幅があるY座標にマークを付ける
    marked_image = cropped_image.copy()
    cv2.line(marked_image, (target_min_x, max_width_y),
             (target_max_x, max_width_y), (0, 255, 0), 2)
    # 画像保存
    cv2.imwrite('./sample_images/marked_image_with_max_width.png', marked_image)
    print("画像を保存しました。")


# 画像のパスを指定して処理を実行
# 画像のパスを適切に指定してください
image_path = './sample_images/M5_DRILL_EXP10000_R.png'
find_max_width(image_path)

最大横幅: 117
画像を保存しました。


検知した幅の一覧を表示する


In [7]:
import cv2
import numpy as np


def find_max_width(image_path):
    # 画像を読み込む
    original_image = cv2.imread(image_path)

    # 画像を反時計回りに90度回転
    rotateimg = cv2.rotate(original_image, cv2.ROTATE_90_COUNTERCLOCKWISE)

    start_y = 160
    end_y = 620 + start_y
    start_x = 280
    end_x = 380 + start_x
    # 画像を指定した範囲で切り取る
    cropped_image = rotateimg[start_y:end_y, start_x:end_x]

    # 画像を2値化する
    gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY)

    # 最大横幅とそのY座標を初期化
    max_width = 0
    max_width_y = 0

    target_min_x = 0
    target_max_x = 0

    # 1行ずつ黒色の最小と最大のピクセルを探し、横幅を計算する
    for y in range(binary_image.shape[0]):
        black_pixels = np.where(binary_image[y, :] == 0)[0]
        if len(black_pixels) > 0:
            min_x = np.min(black_pixels)
            max_x = np.max(black_pixels)
            width = max_x - min_x
            if width > max_width:
                target_max_x = max_x
                target_min_x = min_x
                max_width = width
                max_width_y = y

    print("最大横幅:", max_width)


# 画像のパスを指定して処理を実行

drill_imgs = []
tap_imgs = []
for drill_size in range(3, 7):
    drill_imgs.append(
        f"./sample_images/M{drill_size}_DRILL_EXP10000_R.png")
    drill_imgs.append(
        f"./sample_images/M{drill_size}_DRILL_EXP10000.png")
    tap_imgs.append(
        f"./sample_images/M{drill_size}_TAP_EXP10000_R.png")
    tap_imgs.append(
        f"./sample_images/M{drill_size}_TAP_EXP10000.png")

    for image_path in drill_imgs + tap_imgs:
        print(image_path)
        find_max_width(image_path)

./sample_images/M3_DRILL_EXP10000_R.png
最大横幅: 68
./sample_images/M3_DRILL_EXP10000.png
最大横幅: 68
./sample_images/M3_TAP_EXP10000_R.png
最大横幅: 76
./sample_images/M3_TAP_EXP10000.png
最大横幅: 77
./sample_images/M3_DRILL_EXP10000_R.png
最大横幅: 68
./sample_images/M3_DRILL_EXP10000.png
最大横幅: 68
./sample_images/M4_DRILL_EXP10000_R.png
最大横幅: 92
./sample_images/M4_DRILL_EXP10000.png
最大横幅: 93
./sample_images/M3_TAP_EXP10000_R.png
最大横幅: 76
./sample_images/M3_TAP_EXP10000.png
最大横幅: 77
./sample_images/M4_TAP_EXP10000_R.png
最大横幅: 100
./sample_images/M4_TAP_EXP10000.png
最大横幅: 100
./sample_images/M3_DRILL_EXP10000_R.png
最大横幅: 68
./sample_images/M3_DRILL_EXP10000.png
最大横幅: 68
./sample_images/M4_DRILL_EXP10000_R.png
最大横幅: 92
./sample_images/M4_DRILL_EXP10000.png
最大横幅: 93
./sample_images/M5_DRILL_EXP10000_R.png
最大横幅: 117
./sample_images/M5_DRILL_EXP10000.png
最大横幅: 118
./sample_images/M3_TAP_EXP10000_R.png
最大横幅: 76
./sample_images/M3_TAP_EXP10000.png
最大横幅: 77
./sample_images/M4_TAP_EXP10000_R.png
最大横幅: 100
./sa

In [8]:
def check_tool_type(tool, number):
    M3_DRILL_min = 60
    M3_DRILL_max = 76
    M4_DRILL_min = 84
    M4_DRILL_max = 100
    M5_DRILL_min = 109
    M5_DRILL_max = 125
    M6_DRILL_min = 132
    M6_DRILL_max = 148
    M3_TAP_min = 68
    M3_TAP_max = 85
    M4_TAP_min = 92
    M4_TAP_max = 108
    M5_TAP_min = 124
    M5_TAP_max = 140
    M6_TAP_min = 145
    M6_TAP_max = 161

    if tool == "DRILL":
        if M3_DRILL_min <= number <= M3_DRILL_max:
            return "M3"
        elif M4_DRILL_min <= number <= M4_DRILL_max:
            return "M4"
        elif M5_DRILL_min <= number <= M5_DRILL_max:
            return "M5"
        elif M6_DRILL_min <= number <= M6_DRILL_max:
            return "M6"
    elif tool == "TAP":
        if M3_TAP_min <= number <= M3_TAP_max:
            return "M3"
        elif M4_TAP_min <= number <= M4_TAP_max:
            return "M4"
        elif M5_TAP_min <= number <= M5_TAP_max:
            return "M5"
        elif M6_TAP_min <= number <= M6_TAP_max:
            return "M6"
    return None

# ドリルの種類を調べる
### 工具がドリルかタップか、種類を調べる

 - 工具の一部の部分だけをパターンとして切り出す
 - 工具の写真を撮ったときに、パターンと一致する部分があるかどうかを調べる
 - パターンマッチングで、ドリルかタップかの判別をするのか、穴のサイズを含めた工具の種類を判別するのかを調べる

In [9]:
import cv2


def template_matching(template_path, target_path, threshold=0.8):
    # テンプレート画像の読み込み
    template = cv2.imread(template_path, 0)

    # 対象画像の読み込み
    target = cv2.imread(target_path, 0)
    # 画像を反時計回りに90度回転
    target = cv2.rotate(target, cv2.ROTATE_90_COUNTERCLOCKWISE)

    # テンプレートマッチングの実行
    result = cv2.matchTemplate(target, template, cv2.TM_CCOEFF_NORMED)

    # 最大値とその位置を取得
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    # マッチング率が閾値以上の場合に判定
    if max_val >= threshold:
        # テンプレートが最もマッチした位置に矩形を描画
        h, w = template.shape
        top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        cv2.rectangle(target, top_left, bottom_right, 0, 2)

        # 結果の表示
        cv2.imwrite('Result_templatematching.png', target)
        print("テンプレートが見つかりました。")
    else:
        print("テンプレートは見つかりませんでした。")


if __name__ == "__main__":
    template_path = './tool_templete_tap.png'
    target_path = './sample_images/M6_DRILL_EXP10000_R.png'

    template_matching(template_path, target_path)

テンプレートは見つかりませんでした。


In [10]:
import cv2


def template_matching(template_path, target_path, threshold=0.8):
    # テンプレート画像の読み込み
    template = cv2.imread(template_path, 0)

    # 対象画像の読み込み
    target = cv2.imread(target_path, 0)
    # 画像を反時計回りに90度回転
    target = cv2.rotate(target, cv2.ROTATE_90_COUNTERCLOCKWISE)

    # テンプレートマッチングの実行
    result = cv2.matchTemplate(target, template, cv2.TM_CCOEFF_NORMED)

    # 最大値とその位置を取得
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    # マッチング率が閾値以上の場合に判定
    if max_val >= threshold:
        # テンプレートが最もマッチした位置に矩形を描画
        h, w = template.shape
        top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        cv2.rectangle(target, top_left, bottom_right, 0, 2)

        # 結果の表示
        cv2.imwrite('Result_templatematching.png', target)
        return True
    else:
        return False


if __name__ == "__main__":
    template_path = './tool_templete_tap.png'

    drill_imgs = []
    tap_imgs = []

    for drill_size in range(3, 7):
        drill_imgs.append(
            f"./sample_images/M{drill_size}_DRILL_EXP10000_R.png")
        drill_imgs.append(
            f"./sample_images/M{drill_size}_DRILL_EXP10000.png")
        tap_imgs.append(
            f"./sample_images/M{drill_size}_TAP_EXP10000_R.png")
        tap_imgs.append(
            f"./sample_images/M{drill_size}_TAP_EXP10000.png")

    for drill, tap in zip(drill_imgs, tap_imgs):
        if template_matching(template_path, drill):
            print(f"ドリルを間違えてタップと検知してしまった パス:{drill}")
        else:
            print(drill)
        if not template_matching(template_path, tap):
            print(f"ドリルを検知できなかった パス:{tap}")
        else:
            print(tap)

./sample_images/M3_DRILL_EXP10000_R.png
./sample_images/M3_TAP_EXP10000_R.png
./sample_images/M3_DRILL_EXP10000.png
./sample_images/M3_TAP_EXP10000.png
./sample_images/M4_DRILL_EXP10000_R.png
./sample_images/M4_TAP_EXP10000_R.png
./sample_images/M4_DRILL_EXP10000.png
./sample_images/M4_TAP_EXP10000.png
./sample_images/M5_DRILL_EXP10000_R.png
./sample_images/M5_TAP_EXP10000_R.png
./sample_images/M5_DRILL_EXP10000.png
./sample_images/M5_TAP_EXP10000.png
./sample_images/M6_DRILL_EXP10000_R.png
./sample_images/M6_TAP_EXP10000_R.png
./sample_images/M6_DRILL_EXP10000.png
./sample_images/M6_TAP_EXP10000.png


# 全体図
0. 画像の切り取り
1. 工具の種類の判断
2. 工具径を調べる
3. 工具の突き出し量を調べる
4. 工具を断定する


In [46]:
import cv2
import numpy as np


def setup(image_path, template_path):
    # 画像を読み込む
    original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # テンプレート画像の読み込み
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)

    # 画像を反時計回りに90度回転
    rotateimg = cv2.rotate(original_image, cv2.ROTATE_90_COUNTERCLOCKWISE)

    start_y = 160
    end_y = 620 + start_y
    start_x = 280
    end_x = 380 + start_x
    # 画像を指定した範囲で切り取る
    cropped_image = rotateimg[start_y:end_y, start_x:end_x]

    return cropped_image, template


def template_matching(cropped_image, template, threshold=0.8):
    # テンプレートマッチングの実行
    result = cv2.matchTemplate(cropped_image, template, cv2.TM_CCOEFF_NORMED)

    # 最大値とその位置を取得
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    # マッチング率が閾値以上の場合に判定
    if max_val >= threshold:
        return "TAP"
    else:
        return "DRILL"


def twovalue(cropped_image):
    # 画像を2値化する
    # gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(cropped_image, 128, 255, cv2.THRESH_BINARY)
    max_width = 0
    # 1行ずつ黒色の最小と最大のピクセルを探し、横幅を計算する
    for y in range(binary_image.shape[0]):
        black_pixels = np.where(binary_image[y, :] == 0)[0]
        if len(black_pixels) > 0:
            min_x = np.min(black_pixels)
            max_x = np.max(black_pixels)
            width = max_x - min_x
            if width > max_width:
                target_max_x = max_x
                target_min_x = min_x
                max_width = width
                max_width_y = y
    # 描写
    cv2.line(cropped_image, (target_min_x, max_width_y), (target_max_x, max_width_y), 125, 2)
    return max_width


def check_tool_type(tool, width):
    M3_DRILL_min = 60
    M3_DRILL_max = 76
    M4_DRILL_min = 84
    M4_DRILL_max = 100
    M5_DRILL_min = 109
    M5_DRILL_max = 125
    M6_DRILL_min = 132
    M6_DRILL_max = 148
    M3_TAP_min = 68
    M3_TAP_max = 85
    M4_TAP_min = 92
    M4_TAP_max = 108
    M5_TAP_min = 124
    M5_TAP_max = 140
    M6_TAP_min = 145
    M6_TAP_max = 161

    if tool == "DRILL":
        if M3_DRILL_min <= width <= M3_DRILL_max:
            return "M3"
        elif M4_DRILL_min <= width <= M4_DRILL_max:
            return "M4"
        elif M5_DRILL_min <= width <= M5_DRILL_max:
            return "M5"
        elif M6_DRILL_min <= width <= M6_DRILL_max:
            return "M6"
    elif tool == "TAP":
        if M3_TAP_min <= width <= M3_TAP_max:
            return "M3"
        elif M4_TAP_min <= width <= M4_TAP_max:
            return "M4"
        elif M5_TAP_min <= width <= M5_TAP_max:
            return "M5"
        elif M6_TAP_min <= width <= M6_TAP_max:
            return "M6"
    return None


def find_bottom_black_pixel_coordinates(cropped_image):

    # 画像を2値化する
    _, binary_image = cv2.threshold(cropped_image, 128, 255, cv2.THRESH_BINARY)

    # 画像の高さと幅を取得
    height, width = binary_image.shape

    # 画像を下から上にスキャンして、一番最初に見つかった黒いピクセルの座標を取得
    for y in range(height-1, -1, -1):
        for x in range(width):
            if binary_image[y, x] == 0:  # 黒いピクセルを見つけたら
                return y  # 座標を返す

    # 黒いピクセルが見つからなかった場合
    return None

In [28]:
image_path = './sample_images/M5_DRILL_EXP10000.png'
template_path = './tool_templete_tap.png'
cropped_image, template = setup(image_path, template_path)

tool = template_matching(cv2.rotate(cv2.imread(
    image_path, cv2.IMREAD_GRAYSCALE), cv2.ROTATE_90_COUNTERCLOCKWISE), template)
width = twovalue(cropped_image)
horizon = find_bottom_black_pixel_coordinates(cropped_image)
kekka = check_tool_type(tool, width)
cv2.imwrite("./result_cropped.png", cropped_image)
horizon = 0.037280701754385967*horizon+31.526315789473685


print(f"判定結果 : {kekka}の{tool}  突き出し:{horizon}")
horizon

判定結果 : M5のDRILL  突き出し:54.006578947368425


54.006578947368425

# テキストを表示させる

In [47]:

image_path = './sample_images/M6_TAP_EXP10000.png'
template_path = './tool_templete_tap.png'
cropped_image, template = setup(image_path, template_path)
tool = template_matching(cv2.rotate(cv2.imread(
    image_path, cv2.IMREAD_GRAYSCALE), cv2.ROTATE_90_COUNTERCLOCKWISE), template)
width = twovalue(cropped_image)
kekka = check_tool_type(tool, width)
print(f"判定結果 : {kekka}の{tool}")
width = round(0.037280701754385967*width, 2)
info_text1 = f"Tool Size     : {kekka}"
info_text2 = f"Tool Type     : {tool}"
info_text3 = f"Tool Diameter : {width}"
# テキストを追加
puttext = cv2.putText(cropped_image, info_text1, (1, 500),
                      cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
puttext = cv2.putText(cropped_image, info_text2, (1, 550),
                      cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
puttext = cv2.putText(cropped_image, info_text3, (1, 600),
                      cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

cv2.imwrite("/home/kuga/ソフトウェア/Automatic-Additional-Machinning-System/ImageInspectionController/test/tool_inspection/result_tool.png",cropped_image)

cv2.imwrite("./result_puttext.png", puttext)

判定結果 : M6のTAP


True